In [ ]:
from google.colab import drive
drive.mount("/mntDrive", force_remount=True)

In [ ]:
!pip install numpy==1.17.4

In [ ]:
!pip install tensorflow-gpu==1.15

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

In [ ]:
# %cd drive/My Drive/CarDetection

!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

# !git clone https://github.com/tensorflow/models.git

In [ ]:
%cd /mntDrive/My Drive/CarDetection/models/research

In [ ]:
!pip install tf_slim

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

!python object_detection/builders/model_builder_test.py

In [ ]:
!python setup.py build
!python setup.py install

In [ ]:
%cd ../..

In [ ]:
!python generate_tf_record.py --csv_input=data/train_data.csv --output_path=data/train.record --image_dir=/mntDrive/My Drive/Dataset/KITTI/data_object_image_2/image_2

In [ ]:
!python generate_tf_record.py --csv_input=data/test_data.csv --output_path=data/test.record --image_dir=/mntDrive/My Drive/Dataset/KITTI/data_object_image_2/image_2

In [ ]:
!python generate_tf_record.py --csv_input=data/val_data.csv --output_path=data/val.record --image_dir=/mntDrive/My Drive/Dataset/KITTI/data_object_image_2/image_2

In [ ]:
!pwd

Training of Models

In [4]:
%set_env PYTHONPATH=/mntDrive/My Drive/CarDetection/models/research:/mntDrive/My Drive/CarDetection/models/research/slim

env: PYTHONPATH=/mntDrive/My Drive/CarDetection/models/research:/mntDrive/My Drive/CarDetection/models/research/slim


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [5]:
%cd /mntDrive/My Drive/CarDetection

/mntDrive/My Drive/CarDetection


In [ ]:
%load_ext tensorboard

In [ ]:
import shutil
shutil.copy("/mntDrive/My Drive/Dataset/test.zip", "/content/test.zip")
!unzip /content/test.zip -d /content/test

# SSD MobileNet Object Detector


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz

In [ ]:
!wget https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/ssd_mobilenet_v1_pets.config

In [ ]:
!tar -xf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

In [ ]:
%cd ..

In [ ]:
%tensorflow_version 1.x

In [ ]:
!python models/research/object_detection/model_main.py --logtostderr --model_dir=/mntDrive/My\ Drive/CarDetection/training/ssd_mobilenet_kitti --pipeline_config_path=/mntDrive/My\ Drive/CarDetection/training/ssd_mobilenet.config

In [ ]:
!zip -r /content/model.zip /mntDrive/My\ Drive/CarDetection/data

In [ ]:
!pwd

In [ ]:
!python models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path /mntDrive/My\ Drive/CarDetection/training/ssd_mobilenet.config \
    --trained_checkpoint_prefix /mntDrive/My\ Drive/CarDetection/training/ssd_mobilenet_kitti/model.ckpt-70343 \
    --output_directory /mntDrive/My\ Drive/CarDetection/ssd_model/ssd_kitti_inference_final_2

In [ ]:
#Create TFLite model
!python models/research/object_detection/export_tflite_ssd_graph.py --pipeline_config_path /mntDrive/My\ Drive/CarDetection/training/ssd_mobilenet.config \
    --pipeline_config_path /mntDrive/My\ Drive/CarDetection/training/ssd_mobilenet.config \
    --trained_checkpoint_prefix /mntDrive/My\ Drive/CarDetection/training/ssd_mobilenet_kitti/model.ckpt-70343 \
    --output_directory /mntDrive/My\ Drive/CarDetection/ssd_model/ssd_tflite_final_2 \
    --add_postprocessing_op=true

In [ ]:
!tflite_convert --graph_def_file=/mntDrive/My\ Drive/CarDetection/ssd_model/ssd_tflite_final/tflite_graph.pb \
    --output_file=/mntDrive/My\ Drive/CarDetection/ssd_model/ssd_tflite_final_2/ssd_model_final.tflite \
    --output_format=TFLITE \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
    --inference_type=FLOAT \
    --input_shapes=1,300,300,3 \
    --allow_custom_ops

In [ ]:
import tensorflow as tf
interpreter = tf.lite.Interpreter("/mntDrive/My Drive/CarDetection/ssd_model/ssd_tflite/ssd_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

In [ ]:
%cd /mntDrive/My\ Drive/CarDetection/models/research/

/mntDrive/My Drive/CarDetection/models/research


In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import tensorflow as tf

In [ ]:
base_path = "/mntDrive/My Drive/CarDetection/"
model_name = base_path + "ssd_model/ssd_kitti_inference_final"
path_to_graph = model_name + "/frozen_inference_graph.pb"
path_to_labels = base_path + "training/car_label_map.pbtxt"
num_classes = 1

category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  graph_def = tf.GraphDef()
  with tf.gfile.GFile(path_to_graph, "rb") as fid:
    serialized_graph = fid.read()
    graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(graph_def)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:

    image_tensor = detection_graph.get_tensor_by_name("import/image_tensor:0")
    boxes_tensor = detection_graph.get_tensor_by_name("import/detection_boxes:0")
    scores_tensor = detection_graph.get_tensor_by_name("import/detection_scores:0")
    classes_tensor = detection_graph.get_tensor_by_name("import/detection_classes:0")
    num_detections_tensor = detection_graph.get_tensor_by_name("import/num_detections:0")
  
    for img in os.listdir("/content/test"):
      image_np = cv2.imread(os.path.join("/content/test", img))
      # image_np = image_np.astype(np.float16)
      image_np_exp = np.expand_dims(image_np, axis=0)
      # image_np_exp /= 255.0

      (boxes, scores, classes, num_detections) = sess.run([boxes_tensor, scores_tensor, classes_tensor, num_detections_tensor], feed_dict={image_tensor: image_np_exp})

      boxes = np.squeeze(boxes)
      classes = np.squeeze(classes).astype(np.int32)
      scores = np.squeeze(scores)

      for idx in range(boxes.shape[0]):
        cls = classes[idx]
        score = scores[idx]

        if score > 0.5:
          label = category_index[cls]["name"]
          im_height, im_width, _ = image_np.shape
          ymin, xmin, ymax, xmax = tuple(boxes[idx].tolist())
          xmin = int(xmin * im_width)
          xmax = int(xmax * im_width)
          ymin = int(ymin * im_height)
          ymax = int(ymax * im_height)

          cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
          cv2.putText(image_np, label, (xmin + 10, ymin + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0))

      cv2.imwrite("/content/test_results/" + img, image_np) 


In [ ]:
%tensorboard --logdir "/mntDrive/My Drive/CarDetection/training/ssd_mobilenet_kitti"

# Faster RCNN MobileNet Object Detector

In [ ]:
!python models/research/object_detection/model_main.py --logtostderr --model_dir=/mntDrive/My\ Drive/CarDetection/training/frcnn_mobilenet --pipeline_config_path=/mntDrive/My\ Drive/CarDetection/training/faster_rcnn_mobilenet.config

In [ ]:
%cd /mntDrive/My\ Drive/CarDetection/models/research/

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import tensorflow as tf

In [ ]:
base_path = "/mntDrive/My Drive/CarDetection/"
model_name = base_path + "ssd_model/ssd_kitti_inference_final"
path_to_graph = model_name + "/frozen_inference_graph.pb"
path_to_labels = base_path + "training/car_label_map.pbtxt"
num_classes = 1

category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  graph_def = tf.GraphDef()
  with tf.gfile.GFile(path_to_graph, "rb") as fid:
    serialized_graph = fid.read()
    graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(graph_def)

In [ ]:
%tensorboard --logdir "/mntDrive/My Drive/CarDetection/training/frcnn_mobilenet"

# Faster RCNN Resnet


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz

In [ ]:
!tar -C ./training/frcnn_resnet -xf faster_rcnn_resnet101_coco_2018_01_28.tar.gz

In [ ]:
!python models/research/object_detection/model_main.py --logtostderr --model_dir=/mntDrive/My\ Drive/CarDetection/training/frcnn_resnet_kitti --pipeline_config_path=/mntDrive/My\ Drive/CarDetection/training/faster_rcnn_resnet101_kitti.config

In [ ]:
!python models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path /mntDrive/My\ Drive/CarDetection/training/faster_rcnn_resnet101_kitti.config \
    --trained_checkpoint_prefix /mntDrive/My\ Drive/CarDetection/training/frcnn_resnet_kitti/model.ckpt-174474 \
    --output_directory /mntDrive/My\ Drive/CarDetection/frcnn_resnet_model/

In [ ]:
%cd /mntDrive/My\ Drive/CarDetection/models/research/

/mntDrive/My Drive/CarDetection/models/research


In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import tensorflow as tf

In [ ]:
base_path = "/mntDrive/My Drive/CarDetection/"
model_name = base_path + "frcnn_resnet_model"
path_to_graph = model_name + "/frozen_inference_graph.pb"
path_to_labels = base_path + "training/car_label_map.pbtxt"
num_classes = 1

category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  graph_def = tf.GraphDef()
  with tf.gfile.GFile(path_to_graph, "rb") as fid:
    serialized_graph = fid.read()
    graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(graph_def)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:

    image_tensor = detection_graph.get_tensor_by_name("import/image_tensor:0")
    boxes_tensor = detection_graph.get_tensor_by_name("import/detection_boxes:0")
    scores_tensor = detection_graph.get_tensor_by_name("import/detection_scores:0")
    classes_tensor = detection_graph.get_tensor_by_name("import/detection_classes:0")
    num_detections_tensor = detection_graph.get_tensor_by_name("import/num_detections:0")
  
    for img in os.listdir("/content/test"):
      image_np = cv2.imread(os.path.join("/content/test", img))
      # image_np = image_np.astype(np.float16)
      image_np_exp = np.expand_dims(image_np, axis=0)
      # image_np_exp /= 255.0

      (boxes, scores, classes, num_detections) = sess.run([boxes_tensor, scores_tensor, classes_tensor, num_detections_tensor], feed_dict={image_tensor: image_np_exp})

      boxes = np.squeeze(boxes)
      classes = np.squeeze(classes).astype(np.int32)
      scores = np.squeeze(scores)

      for idx in range(boxes.shape[0]):
        cls = classes[idx]
        score = scores[idx]

        if score > 0.5:
          label = category_index[cls]["name"]
          im_height, im_width, _ = image_np.shape
          ymin, xmin, ymax, xmax = tuple(boxes[idx].tolist())
          xmin = int(xmin * im_width)
          xmax = int(xmax * im_width)
          ymin = int(ymin * im_height)
          ymax = int(ymax * im_height)

          cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
          cv2.putText(image_np, label, (xmin + 10, ymin + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0))

      cv2.imwrite("/content/test_results/" + img, image_np) 

In [ ]:
%tensorboard --logdir "/mntDrive/My Drive/CarDetection/training/frcnn_resnet_kitti"